# Using the cache

First make sure that the dependencies are installed:

```bash
pip install -r requirements-dev.txt
```

Then run redis using docker compose:

```bash
docker compose -f docker-compose.redis.yml up -d
```

Then run the notebook. 

If you want to view the data in redis, you can use the redis-cli:

```bash
brew install redis
redis-cli -p 6380
```

In [1]:
import fsspec
import json
import xarray as xr
from redis import Redis
from redis.asyncio import Redis as RedisAsync
from redis_fsspec_cache.asyncio import RedisAsyncCachingFilesystem
from redis_fsspec_cache.reference import RedisCachingReferenceFileSystem

redis = Redis(host="localhost", port=6380)
redis_async = RedisAsync(host="localhost", port=6380)

s3fs = fsspec.filesystem("s3", anon=True)
cached_s3fs = RedisAsyncCachingFilesystem(redis=redis_async, fs=s3fs, expiry_time=60)

In [2]:
%%time
raw_refs = cached_s3fs.cat_file('s3://nextgen-dmac-cloud-ingest/nos/ngofs2/nos.ngofs2.fields.best.nc.zarr')
refs = json.loads(raw_refs)

new_cached_fs = RedisCachingReferenceFileSystem(
    redis=redis,
    expiry_time=60,
    fo=refs, 
    remote_protocol='s3', 
    remote_options={'anon':True}, 
    target_protocol='s3', 
    target_options={'anon':True}, 
    asynchronous=True, 
)

m = new_cached_fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), 
                      chunks={}, drop_variables=['Itime2'])
ds

CPU times: user 170 ms, sys: 82.3 ms, total: 252 ms
Wall time: 914 ms


<xarray.Dataset> Size: 115GB
Dimensions:             (time: 261, four: 4, nele: 569405, node: 303714,
                         three: 3, maxnode: 10, maxelem: 8, siglev: 41,
                         siglay: 40)
Coordinates:
    lat                 (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    latc                (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    lon                 (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    lonc                (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    siglay              (siglay, node) float32 49MB dask.array<chunksize=(40, 303714), meta=np.ndarray>
    siglev              (siglev, node) float32 50MB dask.array<chunksize=(41, 303714), meta=np.ndarray>
  * time                (time) datetime64[ns] 2kB 2024-04-20T21:00:00 ... 202...
Dimensions without coordinates: four, nele, node, three, maxnode, maxelem
Data variables: (12/42)
    Itime               (time) datetime64[ns] 2kB dask.array<chunksize=(1,), meta=np.ndarray>
    Times               (time) |S26 7kB dask.array<chunksize=(1,), meta=np.ndarray>
    a1u                 (four, nele) float32 9MB dask.array<chunksize=(4, 569405), meta=np.ndarray>
    a2u                 (four, nele) float32 9MB dask.array<chunksize=(4, 569405), meta=np.ndarray>
    art1                (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    art2                (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    ...                  ...
    ww                  (time, siglay, nele) float32 24GB dask.array<chunksize=(1, 40, 569405), meta=np.ndarray>
    x                   (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    xc                  (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    y                   (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    yc                  (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    zeta                (time, node) float32 317MB dask.array<chunksize=(1, 303714), meta=np.ndarray>
Attributes: (12/14)
    Conventions:                 CF-1.0
    CoordinateProjection:        init=nad83:4205
    CoordinateSystem:            GeoReferenced
    GroundWater_Forcing:         GROUND WATER FORCING IS OFF!
    River_Forcing:               THERE ARE 63 RIVERS IN THIS MODEL.\nRIVER IN...
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    ...                          ...
    Tidal_Forcing:               TIDAL ELEVATION FORCING IS OFF!
    history:                     model started at: 20/04/2024   21:15
    institution:                 School for Marine Science and Technology
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    source:                      FVCOM_4.3
    title:                       NGOFS2

In [3]:
%%time
raw_refs = cached_s3fs.cat_file('s3://nextgen-dmac-cloud-ingest/nos/ngofs2/nos.ngofs2.fields.best.nc.zarr')
refs = json.loads(raw_refs)

new_cached_fs = RedisCachingReferenceFileSystem(
    redis=redis,
    expiry_time=60,
    fo=refs, 
    remote_protocol='s3', 
    remote_options={'anon':True}, 
    target_protocol='s3', 
    target_options={'anon':True}, 
    asynchronous=True, 
)

m = new_cached_fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), 
                      chunks={}, drop_variables=['Itime2'])
ds

CPU times: user 36.9 ms, sys: 7.05 ms, total: 43.9 ms
Wall time: 98.9 ms


<xarray.Dataset> Size: 115GB
Dimensions:             (time: 261, four: 4, nele: 569405, node: 303714,
                         three: 3, maxnode: 10, maxelem: 8, siglev: 41,
                         siglay: 40)
Coordinates:
    lat                 (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    latc                (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    lon                 (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    lonc                (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    siglay              (siglay, node) float32 49MB dask.array<chunksize=(40, 303714), meta=np.ndarray>
    siglev              (siglev, node) float32 50MB dask.array<chunksize=(41, 303714), meta=np.ndarray>
  * time                (time) datetime64[ns] 2kB 2024-04-20T21:00:00 ... 202...
Dimensions without coordinates: four, nele, node, three, maxnode, maxelem
Data variables: (12/42)
    Itime               (time) datetime64[ns] 2kB dask.array<chunksize=(1,), meta=np.ndarray>
    Times               (time) |S26 7kB dask.array<chunksize=(1,), meta=np.ndarray>
    a1u                 (four, nele) float32 9MB dask.array<chunksize=(4, 569405), meta=np.ndarray>
    a2u                 (four, nele) float32 9MB dask.array<chunksize=(4, 569405), meta=np.ndarray>
    art1                (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    art2                (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    ...                  ...
    ww                  (time, siglay, nele) float32 24GB dask.array<chunksize=(1, 40, 569405), meta=np.ndarray>
    x                   (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    xc                  (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    y                   (node) float32 1MB dask.array<chunksize=(303714,), meta=np.ndarray>
    yc                  (nele) float32 2MB dask.array<chunksize=(569405,), meta=np.ndarray>
    zeta                (time, node) float32 317MB dask.array<chunksize=(1, 303714), meta=np.ndarray>
Attributes: (12/14)
    Conventions:                 CF-1.0
    CoordinateProjection:        init=nad83:4205
    CoordinateSystem:            GeoReferenced
    GroundWater_Forcing:         GROUND WATER FORCING IS OFF!
    River_Forcing:               THERE ARE 63 RIVERS IN THIS MODEL.\nRIVER IN...
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    ...                          ...
    Tidal_Forcing:               TIDAL ELEVATION FORCING IS OFF!
    history:                     model started at: 20/04/2024   21:15
    institution:                 School for Marine Science and Technology
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    source:                      FVCOM_4.3
    title:                       NGOFS2

In [4]:
%%time
ds.lat.values

CPU times: user 116 ms, sys: 19.3 ms, total: 135 ms
Wall time: 699 ms


array([21.789427, 21.7978  , 21.81    , ..., 30.50772 , 30.50776 ,
       30.507713], dtype=float32)

In [5]:
%%time
ds.lat.values

CPU times: user 2.96 ms, sys: 3.37 ms, total: 6.33 ms
Wall time: 13.3 ms


array([21.789427, 21.7978  , 21.81    , ..., 30.50772 , 30.50776 ,
       30.507713], dtype=float32)

In [6]:
%%time
len(ds.nv.values)

CPU times: user 23 ms, sys: 17.4 ms, total: 40.4 ms
Wall time: 588 ms


3

In [7]:
%%time
len(ds.nv.values)

CPU times: user 6.86 ms, sys: 11 ms, total: 17.8 ms
Wall time: 71.1 ms


3

In [8]:
%%time

ds.zeta.isel(time=slice(0,10)).values

CPU times: user 74.3 ms, sys: 41 ms, total: 115 ms
Wall time: 992 ms


array([[0.1381165 , 0.13736528, 0.13643828, ..., 4.1747184 , 4.163902  ,
        4.177847  ],
       [0.04129069, 0.04099029, 0.04048311, ..., 4.335432  , 4.3234344 ,
        4.3391504 ],
       [0.04776964, 0.04786523, 0.04788679, ..., 4.186504  , 4.1758404 ,
        4.189648  ],
       ...,
       [0.16487393, 0.16326748, 0.16113138, ..., 4.451448  , 4.438706  ,
        4.455572  ],
       [0.16160029, 0.15885413, 0.15340783, ..., 4.4556274 , 4.4428563 ,
        4.459835  ],
       [0.0714745 , 0.06764231, 0.05987399, ..., 4.691659  , 4.677783  ,
        4.6963468 ]], dtype=float32)

In [9]:
%%time

ds.zeta.isel(time=slice(0,10)).values

CPU times: user 17.1 ms, sys: 17.8 ms, total: 34.9 ms
Wall time: 71.1 ms


array([[0.1381165 , 0.13736528, 0.13643828, ..., 4.1747184 , 4.163902  ,
        4.177847  ],
       [0.04129069, 0.04099029, 0.04048311, ..., 4.335432  , 4.3234344 ,
        4.3391504 ],
       [0.04776964, 0.04786523, 0.04788679, ..., 4.186504  , 4.1758404 ,
        4.189648  ],
       ...,
       [0.16487393, 0.16326748, 0.16113138, ..., 4.451448  , 4.438706  ,
        4.455572  ],
       [0.16160029, 0.15885413, 0.15340783, ..., 4.4556274 , 4.4428563 ,
        4.459835  ],
       [0.0714745 , 0.06764231, 0.05987399, ..., 4.691659  , 4.677783  ,
        4.6963468 ]], dtype=float32)